**Task 2**- Submit a GitHub link to a python script that uses LLM/VLM models to caption videos with good accuracy.
Run your script on all 10 videos, share the descriptions it generates, and reflect on how it compares to your manual work. This is where your journey becomes engineering

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers torch torchvision opencv-python Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
import os
import cv2

In [5]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-0

In [13]:
#this is a function to extract a frame from the middle of video
def extract_frame(video_path):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Check if total_frames is valid (greater than 0)
    if total_frames <= 0:
        cap.release()
        print(f"Error: Could not read video or video has no frames: {video_path}")
        return None
    cap.set(cv2.CAP_PROP_POS_FRAMES, total_frames // 2)
    ret, frame = cap.read()
    cap.release()
    if ret:
        # Change the extension to .png
        img_path = video_path.replace(".mp4", "_frame.png").replace(".avi", "_frame.png").replace(".mov", "_frame.png")
        cv2.imwrite(img_path, frame)
        return img_path
    else:
        print(f"Error: Could not read frame from {video_path}")
    return None


In [14]:
def generate_caption(image_path):
    image = Image.open(image_path).convert('RGB')
    inputs = processor(images=image, return_tensors="pt").to(device)
    out = model.generate(**inputs)
    return processor.decode(out[0], skip_special_tokens=True)

In [15]:
#now we will generate captions for all videos
video_dir = "/content/drive/MyDrive/video_samples"
captions = []

for filename in os.listdir(video_dir):
    if filename.endswith((".mp4", ".avi", ".mov")):
        video_path = os.path.join(video_dir, filename)
        print(f"Processing {filename}...")
        frame_path = extract_frame(video_path)
        if frame_path:
            caption = generate_caption(frame_path)
            captions.append(f"{filename}: {caption}")
            print(f"✔️ {filename} → {caption}")
        else:
            print(f"error : Could not extract frame from {filename}")


Processing video1.avi...
✔️ video1.avi → a woman is putting her makeup with a brush
Processing video2.avi...
✔️ video2.avi → a person is cutting a piece of ice
Processing video3.avi...
✔️ video3.avi → a group of people playing a game on a red carpet
Processing video4.avi...
✔️ video4.avi → a man in a kitchen
Processing video5.avi...
✔️ video5.avi → a soccer game is shown in the screen
Processing video6.avi...
✔️ video6.avi → a person is using a keyboard to play music
Processing video7.avi...
✔️ video7.avi → a man is doing a trick on a pole
Processing video8.avi...
✔️ video8.avi → a man is walking a dog on a dirt road
Processing video9.avi...
✔️ video9.avi → a man is writing on a blackboard
Processing video10.avi...
✔️ video10.avi → a man holding a bunch of green leaves


In [16]:
output_path = "/content/drive/MyDrive/captions_output.txt"
with open(output_path, "w") as f:
    for line in captions:
        f.write(line + "\n")

print(f"\nAll video captions are saved to {output_path}")



All video captions are saved to /content/drive/MyDrive/captions_output.txt
